In [ ]:
import sqlalchemy as sql
import random
import pandas as pd

data = pd.read_excel('./data.xlsx')

In [ ]:
# Lista de cidades do RJ
cidades_rj = [
    "Rio de Janeiro", "Niterói", "Petrópolis", "Nova Friburgo", "Teresópolis",
    "Volta Redonda", "Resende", "Barra Mansa", "Macaé", "Campos dos Goytacazes",
    "Angra dos Reis", "Cabo Frio", "Araruama", "São Gonçalo", "Itaboraí",
    "Magé", "Queimados", "Belford Roxo", "Duque de Caxias", "Nova Iguaçu"
]

# Lista de cidades de SP
cidades_sp = [
    "São Paulo", "Campinas", "Santos", "São Bernardo do Campo", "São José dos Campos",
    "Ribeirão Preto", "Sorocaba", "Santo André", "Osasco", "Jundiaí",
    "Bauru", "Piracicaba", "Franca", "Guarulhos", "Diadema",
    "Barueri", "Taboão da Serra", "Mogi das Cruzes", "Taubaté", "Suzano"
]

todas_cidades = cidades_rj + cidades_sp

estados = ['São Paulo', 'Rio de Janeiro']

# inserindo todas as cidades de SP e RJ na coluna 'Cidade'
data['Cidade'] = [random.choice(todas_cidades) for _ in range(data.shape[0])]
data['Estado'] = [random.choice(estados) for _ in range(data.shape[0])]
print(data)

In [ ]:
data['Sigla do estado'] = data['Estado'].apply(lambda x: 'SP' if x == 'São Paulo' else 'RJ')

In [ ]:
data['Idade'] = [random.randint(18, 60) for _ in range(data.shape[0])]

In [ ]:
# Caso o usuario tenha mais de 1000 na conta ele tera um valor aleatorio entre 400 a 2000 em compras
if (data['Saldo na conta'] > 1000).any():
    data['Compras'] = [random.randint(400, 2000) for _ in range(data.shape[0])]
else: # caso o valor seja menor que 1000 ele tera um valor de 10 a 1000 em compras
    data['Compras'] = [random.choice(10, 1000) for _ in range(data.shape[0])]

print(data)

In [ ]:
names = [
    "Ana", "João", "Maria", "Pedro", "Lucas",
    "Beatriz", "Carlos", "Júlia", "Rafael", "Camila",
    "Gabriel", "Fernanda", "Matheus", "Larissa", "Bruno",
    "Mariana", "Henrique", "Luana", "Felipe", "Letícia"
]

data['Name'] = [random.choice(names) for _ in range(data.shape[0])]


In [ ]:
engine = sql.create_engine('sqlite:///:memory:')
conn = engine.connect()

data.to_sql('clientes', conn, index=False, if_exists='replace')

def query(query, connection):
    query = pd.read_sql(query, connection)
    return query


q = '''
    select c."name", c."compras"
        from clientes c
    where c."name" like "L%"
'''
result = query(q, engine)

name_result = result['Name'].value_counts()
total_compras = result.groupby('Name')['Compras'].sum()
data['Total compras'] = data['Compras'].apply(lambda x: f'R$ {x:.2f}')
data_cleaned = data.dropna(axis=1)
print(data_cleaned)


In [ ]:
data_cleaned.to_excel('./data.xlsx', index_label="ID")